In [1]:
import os
import pandas as pd
import json

In [6]:
# Get games for a league
games = pd.read_json("game-changers/esports-data/mapping_data.json")

# Open target game file
target_game = "0a63934c-9907-4b7c-a553-ac945cc9eea4"

try:
        with open(os.path.join('game-changers/games/2022/', 'val_'+target_game+'.json'), 'r') as file:

                game_data = json.load(file)
except:
        print("No such file")

In [7]:
# Get player mapping to connect player id 
session_players = games[games['platformGameId'] == "val:"+target_game]['participantMapping']
session_players = session_players.to_dict()[session_players.index[0]]
session_players

{'1': '108764514258014973',
 '2': '108764514005333971',
 '3': '108764513807403457',
 '4': '108764522500018946',
 '5': '108764516777284603',
 '6': '109030080591889737',
 '7': '109030079523193762',
 '8': '109030082839148195',
 '9': '109030083962930086',
 '10': '109030081641666982'}

In [9]:
# Get event count

event_count = {}

for snapshot in game_data:

    event_type = list(snapshot.keys())[1]
    #print(event_type)

    if event_type in event_count:
        event_count[event_type] += 1
    else:
        event_count[event_type] = 1

pd.Series(event_count)

snapshot                1320
configuration            330
observerTarget          3562
metadata                1979
gamePhase                 94
playerSpawn              230
roundStarted              23
spikeStatus              163
inventoryTransaction     902
damageEvent             1277
playerDied               159
roundDecided              23
roundCeremony             23
roundEnded                22
timeout                    6
gameDecided                1
dtype: int64

# Data Mining
---
Refer to the hackathon [data source docs](https://docs.google.com/document/d/19H3FsWYEli6ShIV5_5fmhq2xDdfcQOCXlRiNvKZLYL8/edit#heading=h.lkyo8c40dcz1) for the structure for each event

In [12]:
def getEvents(event_type):

    type_events = []

    for event in game_data:
        
        if list(event.keys())[1]  == event_type:
            type_events.append(event)
    
    return type_events

### Most Damage Dealt
---

In [23]:
damageEvents = getEvents("damageEvent")

damageEvents[0]

total_dmg_for_game = {}

for event in damageEvents:

    causer = session_players[str(event['damageEvent']['causerId']['value'])]
    if causer in total_dmg_for_game:

        total_dmg_for_game[causer] += event['damageEvent']['damageAmount']

    else:
        total_dmg_for_game[causer] = event['damageEvent']['damageAmount']

pd.Series(total_dmg_for_game)

108764516777284603    3316.499673
108764514005333971    2701.137631
109030079523193762    1455.425219
108764513807403457    2520.555618
109030083962930086    2761.903740
108764514258014973    2583.819286
109030080591889737    2596.611325
108764522500018946    2534.260872
109030082839148195    2086.288773
109030081641666982    2860.406178
dtype: float64

### Target Player Hitbox Hits

In [25]:
damageEvents = getEvents("damageEvent")

hitbox_hit_count = {'body': 0, 'head': 0, 'general': 0}

interested_causer = 5

for event in damageEvents:

    # If body or head shot or general
    hitbox_location =  event['damageEvent']['location']
    
    if hitbox_location == "BODY":
        hitbox_hit_count['body'] += 1
    elif hitbox_location == "HEAD":
        hitbox_hit_count['head'] += 1
    else:   # General and others
        hitbox_hit_count['general'] += 1
        

pd.Series(hitbox_hit_count)

body       430
head        98
general    749
dtype: int64

### Player with most lethal hits

In [29]:
damageEvents = getEvents("damageEvent")

damageEvents[0]

lethal_hit_count = {}

for event in damageEvents:

    causer = session_players[str(event['damageEvent']['causerId']['value'])]

    if event['damageEvent']['killEvent']:

        if causer in lethal_hit_count:

            lethal_hit_count[causer] += 1

        else:
            lethal_hit_count[causer] = 1

pd.Series(lethal_hit_count)

108764514005333971    19
109030083962930086    20
109030080591889737    14
108764513807403457    15
109030079523193762     8
108764516777284603    19
108764514258014973    21
109030081641666982    20
108764522500018946    17
109030082839148195    11
dtype: int64